In [ ]:
import pandas as pd
import ahocorasick

%load_ext autoreload

%autoreload 2

In [ ]:
df = pd.read_csv('/Users/kylegreen/codeup-data-science/trials_in_rust/trials-in-rust/first_rust_program/Testing/Almustaqbal.csv')
df.head()

In [ ]:
df.head(1).headline.values

In [ ]:
df.head(1).text

In [ ]:
df.head(1).text.values

In [ ]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline

In [ ]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline

#mix = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment")
#ca = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-ca-sentiment")
#da = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
msa = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")

#half = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-half")
#quarter = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-quarter")
#eighth = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-eighth")
#sixteenth = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sixteenth")


In [ ]:

models = (mix, ca, da, msa, half, quarter, eighth, sixteenth)

In [ ]:
for mod in models:
    print(mod('hello'))

In [ ]:
df.head(5).text.values

In [ ]:
sum([len(val.split()) for val in df.head(100).text.values])/100
modelable = df.head(2000).text[[True if len(val.split()) < 350 else False for val in df.head(2000).text.values]]
for val in modelable.values:
    print(msa(val))


In [ ]:
for val in df.head(200).values:
    try:
        print(msa(val))
    except:
        continue
    
    #for mod in models:
        #print(mod(val[:512]))
        
subset = df.head(100).copy()

        
        
subset['label'] = subset.apply(lambda x:  msa(x.text)[0]['label'] if len(x.text.split()) < 300 else False, axis=1)
subset['score'] = subset.apply(lambda x: msa(x.text)[0]['score'] if len(x.text.split()) < 300 else False, axis=1)
subset.head()

#texts = {}    
#for i, val in enumerate(df.head(200).text.values):
#    if len(val) > 35 and len(val) < 512:
#        texts[i] = val
        
        
#print(texts)
        
        

In [ ]:
subset.head(15)

In [ ]:
subset.score.groupby(subset.label).max()

In [ ]:
subset[subset.label== 'negative'].max()

In [ ]:
translator = Translator()

In [ ]:
def translate_to_english(text):
    translation = translator.translate(text, dest='en').text
    return translation

translate_to_english(subset.head(1).text)

In [ ]:
subset.to_csv('labels.csv', index=False)

In [ ]:
sub = pd.read_csv('labels.csv')
sub.head()

In [ ]:
translator = Translator()
def translate_to_english(text):
    translation = translator.translate(text, dest='en').text
    return translation

def check_for_america(translation):
    if 'america' in translation.lower() or 'united states' in translation.lower():
        return True
    else:
        return False

text = sub.head(1).text



#translation = translate_to_english(text)
#check_for_america(translation)

row_nums = df.shape[0]
sets_of_ten = int(row_nums / 100)

collection_of_dfs = []
print(sets_of_ten)

for i in range(0,100):
    collection_of_dfs.append(df[df.index.isin(range(i * sets_of_ten, (i+1) * sets_of_ten))].copy())
    collection_of_dfs[i]['america'] = [True if val.str.contains('')]
    print(collection_of_dfs[f'frame_{i}'])
    break
    

#sub['en_text'] = sub.apply(lambda x: translate_to_english(x.text), axis=1)
#sub.head(10)

In [ ]:
df['source'] = 'almustaqbal'

In [ ]:
df[mask].to_csv('almustaqbal.csv')

In [ ]:
def flip_key_value_pairs(dicts):
    res = dict((v,k) for k,v in dicts.items())
    return res

def using_ahocorasick(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    tags = col.apply(lambda x: list(A.iter(x)))
    return mask, tags

def look_for_words_in_text(df_text):
    
    topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}
    
    topics = flip_key_value_pairs(topics)
    
    tags = []
    for key in topics.keys():
        if key in df_text:
            tags.append(key)
            
    return tags

def make_relevant_tagged_df(df):
    topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}


    topics = flip_key_value_pairs(topics)
    mask, tags = using_ahocorasick(df.text, list(topics.keys()))

    copied = df[mask].copy()

    copied['tags'] = copied.text.apply(look_for_words_in_text)

    return copied

publication_names = ['Alittihad', 'Almasryalyoum', 'Almustaqbal', 'Alqabas', 'Echoroukonline', 'Ryiadh', 'Sabanews', 'SaudiYoum', 'Techreen', 'Youm7']

filepath = '/Users/kylegreen/codeup-data-science/trials_in_rust/trials-in-rust/first_rust_program/Testing/'

for name in publication_names:
    path = filepath+name
    df = pd.read_csv(path + '.csv')
    df = make_relevant_tagged_df(df)
    df['source'] = name
    df.to_csv(name+'.csv', index=False)
    print(name)
print('done')

In [ ]:
for val in copied.tags.values:
    if len(val) > 1:
        print(val)

In [ ]:
sub['america'] = sub.apply(lambda x: check_for_america(x.en_text), axis=1)
sub.head(15)

In [ ]:
sub[sub.america==True].label.value_counts()

In [ ]:
pd.read_csv(publication_names[4] + '.csv')

In [ ]:
csvs = []
for name in publication_names:
    csvs.append(pd.read_csv(name + '.csv'))
df = pd.concat(csvs)
df.to_csv('all_articles.csv', index=False)

In [ ]:
df.source.value_counts()

In [ ]:
df.shape

In [ ]:
df.sample(500).to_csv('sample_articles.csv', index=False)

In [ ]:
steps = int(df.shape[0] /3) 

In [ ]:
split_dfs = []
for i in range(0, df.shape[0] - steps, steps):
    split_dfs.append(df.iloc[i: i+steps])
    
split_dfs[0].text.unique().size
    

In [ ]:
print(i)

In [ ]:
for i, d in enumerate(split_dfs):
    d.to_csv(f'block_{i+1}.csv', index=False)

In [ ]:
split_dfs[2].head()

In [ ]:
summarizer = pipeline('summarization')

In [ ]:
def make_msa(df_text):
    try:
        done = msa(df_text)
        return done
    except:
        try:
            first_half = msa(df_text[:round(len(df_text)/2)]) 
            second_half = msa(df_text[round(len(df_text)/2):])
            if first_half[0]['label'] == second_half[0]['label']:
                label = first_half[0]['label']
                score = (test_1[0]['score'] + test_2[0]['score'])/2
                done = [{'label': label, 'score': score}]
                return done
        except:
            print(df_text)
            return False
        

def analyze_text(df):
    scores = []
    scores = df.iloc[0:50].text.apply(make_msa)
    #df['score'] = df.text.apply(lambda x: msa(x)[0]['score'] if len(x.split()) < 300 else False)
    return scores

def analyze_headline(df):
    headline_scores = []
    scores = df.iloc[0:200].headline.apply(make_msa)
    return scores

#(analyze_text(df) != False).sum()
(analyze_text(subset_df) != False).sum()

In [ ]:
text= 'ائيل عن وثائق جديدة اليوم، السبت، عن أسرار حرب الاستنزاف التى خاضتها مصر ض'

In [ ]:
subset_df = pd.read_csv('sample_articles.csv')
subset_df.head()

In [ ]:
df.iloc[0]

In [ ]:
for val in subset_df.text.values:
    if text in val:
        print(subset_df[subset_df.text==val])

In [ ]:
source_dicts = {}
for source in df.source.unique():
    source_dicts[source] = df[df.source == source]
    
source_dicts['SaudiYoum']
    

In [ ]:
import re

In [ ]:
df.dateline.values[0]

In [ ]:
for val in list(df.dateline.values):
    try:
        print(re.search('20\d\d\W\d\d\W\d\d', val).group())
        
    except:
        print(val)

#print(len(df.dateline.values))

In [ ]:
source_dicts['SaudiYoum'].dateline.values[0]

In [ ]:


def using_ahocorasick_months(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    return mask

def look_for_words_in_text_months(df_text):
    
    months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
    months = flip_key_value_pairs(months)
    
    tags = []
    for key in months.keys():
        if key in df_text:
            tags.append(key)
            
    return tags

mask = using_ahocorasick(df.dateline, list(months.values()))[0]
copy = df[mask].copy()
copy = pd.DataFrame(copy)
copy['month'] = copy.dateline.apply(look_for_words_in_text_months)


In [ ]:
copy

In [ ]:
def find_month(df_dateline):
    exp = '\d\d.*\d\d\d\d'
    try:
        month = re.search(exp, df_dateline).group()[2:-4]
        months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
        months = flip_key_value_pairs(months)
        
        return re.search(exp, df_dateline).group()[2:-4]
    except:
        
        try:
            return re.search('20\d\d\W\d\d\W\d\d', val).group()

        except:
            return False

df.dateline.apply(find_month)

In [ ]:
df.dateline.size

In [ ]:
df=pd.read_csv('all_articles.csv')

In [2]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline


""" 
Change name to which file you're doing sentiment analysis on, NOT including the '.csv' 
Output file will be 'labeled_<filename>.csv'
"""

msa = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")



def load_and_label_df(name):
    path = 'split_articles/'
    df = load_csv(path + name)
    print(f'loaded {name}')
    print('labeling/scoring...')
    df = create_labels_scores(df, name)
    print('done labeling/scoring!')
    return df

def load_csv(filename):
    df = pd.read_csv(filename)
    return df

def make_msa(df_text):
    
    try:
        done = msa(df_text)
        return done
    except:
        
        try:
            first_half = msa(df_text[:round(len(df_text)/2)]) 
            second_half = msa(df_text[round(len(df_text)/2):])
            if first_half[0]['label'] == second_half[0]['label']:
                label = first_half[0]['label']
                score = (test_1[0]['score'] + test_2[0]['score'])/2
            done = [{'label': label, 'score': score}]
            return done
        except:
        
            try:
                beginning = msa(df_text[:round(len(df_text)/3)]) 
                middle = msa(df_text[round(len(df_text)/3):round(len(df_text)*2/3)])
                end = msa(df_text[round(len(df_text)*2/3):])

                if (beginning[0]['label'] == middle[0]['label']) and (beginning[0]['label'] == end[0]['label']):
                    label = first_half[0]['label']
                    score = (beginning[0]['score'] + middle[0]['score'] + end[0]['score'])/3
                    done = [{'label': label, 'score': score}]
                    return done
                else:
                    return 'sentiment of parts not equal'
            except:
                return '3 is not enough'
        
def analyze_text(df):
    scores = []
    print('analyzing_texts')
    scores = [make_msa(val) for val in df.text.values]
    return scores

def analyze_headline(df):
    print('analyzing headlines')
    headline_scores = [make_msa(val) for val in df.headline.values]
    return headline_scores

def label_and_scores(msa_scores):
    labels = []
    scores = []
    for val in msa_scores:
        try:
            labels.append(val[0]['label'])
            scores.append(val[0]['score'])
        except:
            labels.append(False)
            scores.append(False)

    return labels, scores

def create_labels_scores(df, name):
    text_scores = analyze_text(df)
    labels, scores = label_and_scores(text_scores)
    df['text_label'] = labels
    df['text_score'] = scores

    headline_scores = analyze_headline(df)
    labels, scores = label_and_scores(headline_scores)
    df['headline_label'] = labels
    df['headline_score'] = scores

    # CHANGE 'BLOCK_NAME' TO WHATEVER YOU WANT
    df.to_csv('labeled_split_articles/labeled_'+ name + '.csv', index=False)

    return df

#load_and_label_df(name)




In [ ]:
pd.read_csv('labeled_' +name+'.csv')

In [ ]:
def split_tons_of_csvs():
    df = pd.read_csv('/Users/kylegreen/codeup-data-science/all_articles.csv')
    
    end_point = '/Users/kylegreen/codeup-data-science/arabic_media_nlp_project/kyles_work/split_articles/'
    
    df_shape = df.shape[0]
    
    one_thousandth = int(df_shape/1000)
    
    for i in range(0, df_shape, one_thousandth):
        copy = df.iloc[i:i+one_thousandth].copy()
        copy.to_csv(end_point+str(i) + '.csv', index=False)
    
split_tons_of_csvs().head()

In [ ]:
pd.read_csv('split_articles/354501.csv')

In [ ]:
import os
import regex as re
import pandas as pd

labeled = 'labeled_split_articles'
unlabeled = 'split_articles'

os.listdir(labeled)

unlabeled_files = os.listdir(unlabeled)
labeled_files = []

for path in os.listdir(labeled):
    labeled_files.append(re.search('\d*\.', path).group()[:-1])

print(labeled_files)
    
    
ul_files = unlabeled_files[:]
files_found  = []
for p in labeled_files:
    for pth in unlabeled_files:
        if p+'.csv' == pth:
            ul_files.remove(pth)

path = 'split_articles/'
for file in ul_files:
    load_and_label_df(file)
    


['328083', '274890', '167076', '188853', '326655', '194922', '272748', '39627', '48552', '354501', '158508', '207060', '11424', '236334', '81396', '20349', '267393', '50337', '338079', '72828', '191709', '33915', '6783', '103530', '268464', '91392', '79968', '193137', '261324', '184926', '261681', '18207', '240261', '121023', '17493', '74256', '154938', '264894', '228123', '336651', '51765', '158865', '25347', '89250', '168147', '149940', '353787', '277746', '130662', '201348', '343434', '347718', '297738', '26061', '325941', '162078', '94605', '357', '12138', '124950', '142800', '334866', '187425', '234906', '327012', '186711', '35343', '225981', '54621', '121380']
loaded 255255.csv
labeling/scoring...
analyzing_texts
analyzing headlines
done labeling/scoring!
loaded 131019.csv
labeling/scoring...
analyzing_texts
analyzing headlines
done labeling/scoring!
loaded 120666.csv
labeling/scoring...
analyzing_texts
analyzing headlines
done labeling/scoring!
loaded 271320.csv
labeling/scoring

In [ ]:
import os
import pandas as pd


labeled = 'labeled_split_articles'



bunk_files = []

for fi in os.listdir(labeled):
    df = pd.read_csv(labeled + '/' + fi)
    if df.text_score.sum() == 0:
        bunk_files.append(fi)
        
for fi in bunk_files:
    if os.path.isfile(f'{labeled}/{fi}'):
        os.remove(f'{labeled}/{fi}')
    else:
        print(f"{fi} doesn't exist")
        

In [ ]:
import regex as re
import pandas as pd

def find_month(df_dateline):
    exp = '\d\d.*\d\d\d\d'
    try:
        month = re.search(exp, df_dateline).group()[2:-4]
        months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
        months = flip_key_value_pairs(months)
        
        return re.search(exp, df_dateline).group()[2:-4]
    except:
        
        try:
            return re.search('20\d\d\W\d\d\W\d\d', val).group()

        except:
            return False


def flip_key_value_pairs(dicts):
    res = dict((v,k) for k,v in dicts.items())
    return res

df.head()

In [ ]:
pd.read_csv('labeled_split_articles/labeled_312732.csv.csv')

In [ ]:
path = '/Users/kylegreen/codeup-data-science/arabic_media_nlp_project/kyles_work/labeled_split_articles'
name = 'labeled_187425.csv.csv'
filepath = path+'/'+name

pd.read_csv(filepath)
